#Vancouver Crime Analysis Program
#####Author: Luke Hansen
---
<b>What - </b>This notebook will report the top crimes in popular neighbourhoods in Vancouver, predict the number of certain crimes, and attempt to find a corelation between crime rates and housing values.

---
<b>Why - </b>The goal of this project is to teach myself:
- basic big data principles
- Python 3
- Apache Spark
- Apache Kafka
- the databricks platform

---

<b>How - </b>Datasets Used:

The Vancouver Police Department's CSV dataset (2003 - present): https://data.vancouver.ca/datacatalogue/crime-data.htm

Vancouver property tax datasets (2006 - 2018): https://data.vancouver.ca/datacatalogue/propertyTax.htm

---

<b>Conclusions - </b> Final speculations:

The higher the property market, the higher the crime there will be. 

The lower the property market, the lower the crime will be.

#####The main speculation that I've made is that the crime rate goes up and down with the market due to affordability for housing, as people can be pushed to crime if they cannot afford housing.

---

####Content:
1. Crime in Vancouver
2. Housing Values in Vancouver
3. Corelations Between Crime Rates and Housing Values

##1. Crime in Vancouver

The first task to be completed is to import the crime data from the DBFS into the notebook (NOTE: I plan to use Apache Kafka to fetch the data real-time on each execution of this notebook, but for now I'll just manually download the datasets individually and upload them to Databricks).

In [4]:
#todo connect apache kafka to fetch data in real time

# import crime csv file from DBFS
crime_path = "/FileStore/tables/crime_csv_all_years.csv"
crime_dataset = spark.read.csv(crime_path, inferSchema=True, header=True)

crime_dataset.show(20)

+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+
 TYPE|YEAR|MONTH|DAY|HOUR|MINUTE| HUNDRED_BLOCK| NEIGHBOURHOOD| X| Y|
+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+
 Theft from Vehicle|2003| 5| 14| 0| 0| 11XX CONNAUGHT DR| Shaughnessy|490368.39|5454609.25|
Break and Enter R...|2003| 5| 19| 22| 30| 18XX COMOX ST| West End|489813.77|5459590.38|
 Theft from Vehicle|2003| 1| 17| 21| 30| 21XX YORK AVE| Kitsilano|488818.32| 5457672.4|
 Theft from Vehicle|2003| 1| 17| 18| 30| 21XX YORK AVE| Kitsilano|488818.32| 5457672.4|
 Theft from Vehicle|2003| 5| 17| 17| 0| 6XX BURRARD ST|Central Business ...|491279.21|5459177.42|
Break and Enter R...|2003| 1| 18| 9| 0| 33XX E 2ND AVE| Hastings-Sunrise|497666.36|5457305.02|
Offence Against a...|2003| 11| 1|null| null|OFFSET TO PROTECT...| null| 0.0| 0.0|
Break and Enter R...|2003| 7| 7| 11| 0| 66XX MAIN ST| Sunset|492601.32|5452351.02|
 Theft from Vehicle|2003| 1| 10| 1| 0| 21XX YORK AVE| Kitsilano|488818.32| 5457672.4|
 Theft from Vehicle|2003| 11| 3| 17| 55|BURRARD ST / COMO...| West End|490757.46|5458643.86|
Offence Against a...|2003| 4| 27|null| null|OFFSET TO PROTECT...| null| 0.0| 0.0|
 Theft from Vehicle|2003| 3| 11| 21| 0| 21XX WAVERLEY AVE| Victoria-Fraserview|495350.96|5452706.74|
Break and Enter R...|2003| 11| 17| 19| 20| 33XX E 2ND AVE| Hastings-Sunrise|497708.07|5457295.56|
Offence Against a...|2003| 11| 4|null| null|OFFSET TO PROTECT...| null| 0.0| 0.0|
Break and Enter R...|2003| 10| 17| 15| 30| 18XX COTTON DR| Grandview-Woodland|494787.03|5457299.84|
Break and Enter R...|2003| 6| 4| 15| 30| 18XX COTTON DR| Grandview-Woodland|494796.22|5457303.43|
 Theft from Vehicle|2003| 7| 11| 19| 0| 21XX WATERSIDE AVE| Victoria-Fraserview|495310.45|5450562.68|
Break and Enter R...|2003| 2| 20| 11| 30| 33XX E 3RD AVE| Hastings-Sunrise|497571.63|5457204.76|
Break and Enter C...|2003| 9| 26| 2| 30| 10XX ALBERNI ST| West End|491067.65|5459114.22|
Offence Against a...|2003| 8| 30|null| null|OFFSET TO PROTECT...| null| 0.0| 0.0|
+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+
only showing top 20 rows

Due to privacy concerns, the Vancouver Police Department have nullified the time and place of any violent records, so I won't include those for these neighbourhood-specific records.

In [6]:
# trim away NULL values
full_crime_df = crime_dataset
crime_df = crime_dataset.orderBy("YEAR").filter(crime_dataset["NEIGHBOURHOOD"]!="null")
crime_df.show(20)

+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+
 TYPE|YEAR|MONTH|DAY|HOUR|MINUTE| HUNDRED_BLOCK| NEIGHBOURHOOD| X| Y|
+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+
 Theft from Vehicle|2003| 3| 3| 21| 0| 21XX WATERLOO ST| Kitsilano|486927.58|5457222.16|
Break and Enter R...|2003| 10| 17| 15| 30| 18XX COTTON DR| Grandview-Woodland|494787.03|5457299.84|
 Theft of Vehicle|2003| 6| 6| 15| 0| 32XX COMMERCIAL DR|Kensington-Cedar ...|494950.33|5455888.51|
Break and Enter R...|2003| 7| 7| 11| 0| 66XX MAIN ST| Sunset|492601.32|5452351.02|
Break and Enter R...|2003| 11| 17| 19| 20| 33XX E 2ND AVE| Hastings-Sunrise|497708.07|5457295.56|
Break and Enter R...|2003| 2| 20| 11| 30| 33XX E 3RD AVE| Hastings-Sunrise|497571.63|5457204.76|
 Theft from Vehicle|2003| 1| 17| 16| 30| 6XX BURRARD ST|Central Business ...|491279.21|5459177.42|
 Theft from Vehicle|2003| 1| 17| 21| 30| 21XX YORK AVE| Kitsilano|488818.32| 5457672.4|
Break and Enter R...|2003| 1| 18| 9| 0| 33XX E 2ND AVE| Hastings-Sunrise|497666.36|5457305.02|
 Theft from Vehicle|2003| 11| 3| 17| 55|BURRARD ST / COMO...| West End|490757.46|5458643.86|
 Theft from Vehicle|2003| 3| 11| 21| 0| 21XX WAVERLEY AVE| Victoria-Fraserview|495350.96|5452706.74|
Break and Enter R...|2003| 6| 4| 15| 30| 18XX COTTON DR| Grandview-Woodland|494796.22|5457303.43|
 Theft from Vehicle|2003| 7| 11| 19| 0| 21XX WATERSIDE AVE| Victoria-Fraserview|495310.45|5450562.68|
Break and Enter C...|2003| 9| 26| 2| 30| 10XX ALBERNI ST| West End|491067.65|5459114.22|
 Theft from Vehicle|2003| 6| 24| 20| 0| 21XX WATERSIDE AVE| Victoria-Fraserview|495310.45|5450562.68|
 Theft from Vehicle|2003| 5| 14| 0| 0| 11XX CONNAUGHT DR| Shaughnessy|490368.39|5454609.25|
Break and Enter R...|2003| 5| 19| 22| 30| 18XX COMOX ST| West End|489813.77|5459590.38|
 Theft from Vehicle|2003| 1| 17| 18| 30| 21XX YORK AVE| Kitsilano|488818.32| 5457672.4|
 Theft from Vehicle|2003| 5| 17| 17| 0| 6XX BURRARD ST|Central Business ...|491279.21|5459177.42|
 Theft from Vehicle|2003| 1| 10| 1| 0| 21XX YORK AVE| Kitsilano|488818.32| 5457672.4|
+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+
only showing top 20 rows

####Widget Control
This modified dataframe can now be used for the neighbourhood widget, so that the user can dynamically control and explore different neighbourhoods and their crime statistics.

In [8]:
# widget control
neighbourhoods_list = crime_df.select("NEIGHBOURHOOD").distinct().rdd.flatMap(lambda x: x).collect()
dbutils.widgets.dropdown("neighbourhood", "Kitsilano", neighbourhoods_list)

####Number of Crimes in Area
The crimes are now specific to the neighbourhood variable that's currrently selected, and a count can be queried.

In [10]:
crimes = crime_df.filter(crime_df["NEIGHBOURHOOD"]==dbutils.widgets.get("neighbourhood"))

# count the number of crimes in the chosen neighbourhood
num_crimes = crimes.count()
print("There have been " + str(num_crimes) + " crimes in " + str(dbutils.widgets.get("neighbourhood")) + " since the year 2003.")

There have been 6185 crimes in Shaughnessy since the year 2003.

####Linear graph plotting the amount of crimes per year in this area from 2003 - 2018

In [12]:
# number of crimes each year since 2003 in the selected neighbourhood
crimes_per_year = crimes.groupBy("YEAR").count().orderBy("count").filter(crime_dataset["YEAR"]!="2019")
display(crimes_per_year)

YEAR,count
2009,239
2010,257
2008,271
2013,275
2007,334
2006,336
2012,349
2011,351
2015,371
2014,376


####Sum of Crimes in this Area Since 2003

In [14]:
from pyspark.sql.functions import desc
top_crimes = crimes.groupBy('TYPE').count().orderBy("count").sort(desc("count"))
display(top_crimes)

TYPE,count
Theft from Vehicle,2086
Break and Enter Residential/Other,2000
Mischief,717
Vehicle Collision or Pedestrian Struck (with Injury),650
Theft of Vehicle,389
Theft of Bicycle,159
Break and Enter Commercial,150
Other Theft,27
Vehicle Collision or Pedestrian Struck (with Fatality),7


####All of Vancouver
Now we can see what the sum of all of the crimes - including violent crimes - looks like.

A new DATE column can be added by just concatenating the year, month and day columns, and then casting to TimeStamp objects.

In [16]:
from pyspark.sql.functions import concat, col, lit
from pyspark.sql import functions as sf
import pyspark.sql

full_crime_df = full_crime_df.withColumn('DATE',
    sf.from_unixtime(sf.unix_timestamp(sf.concat(sf.col('YEAR'), sf.lit('-'), sf.col('MONTH'), sf.lit('-'), sf.col("DAY")), 'yyyy-MM-dd')).cast("timestamp"))

In [17]:
data = full_crime_df.filter(full_crime_df["YEAR"] != "2019")
display(data)

TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,DATE
Theft from Vehicle,2003,5,14,0,0,11XX CONNAUGHT DR,Shaughnessy,490368.39,5454609.25,2003-05-14T00:00:00.000+0000
Break and Enter Residential/Other,2003,5,19,22,30,18XX COMOX ST,West End,489813.77,5459590.38,2003-05-19T00:00:00.000+0000
Theft from Vehicle,2003,1,17,21,30,21XX YORK AVE,Kitsilano,488818.32,5457672.4,2003-01-17T00:00:00.000+0000
Theft from Vehicle,2003,1,17,18,30,21XX YORK AVE,Kitsilano,488818.32,5457672.4,2003-01-17T00:00:00.000+0000
Theft from Vehicle,2003,5,17,17,0,6XX BURRARD ST,Central Business District,491279.21,5459177.42,2003-05-17T00:00:00.000+0000
Break and Enter Residential/Other,2003,1,18,9,0,33XX E 2ND AVE,Hastings-Sunrise,497666.36,5457305.02,2003-01-18T00:00:00.000+0000
Offence Against a Person,2003,11,1,null,null,OFFSET TO PROTECT PRIVACY,null,0.0,0.0,2003-11-01T00:00:00.000+0000
Break and Enter Residential/Other,2003,7,7,11,0,66XX MAIN ST,Sunset,492601.32,5452351.02,2003-07-07T00:00:00.000+0000
Theft from Vehicle,2003,1,10,1,0,21XX YORK AVE,Kitsilano,488818.32,5457672.4,2003-01-10T00:00:00.000+0000
Theft from Vehicle,2003,11,3,17,55,BURRARD ST / COMOX ST,West End,490757.46,5458643.86,2003-11-03T00:00:00.000+0000


####Sum of crimes in all of Vancouver since 2003

In [19]:
top_crimes = data.groupBy('TYPE').count().orderBy("count").sort(desc("count"))
display(top_crimes)

TYPE,count
Theft from Vehicle,194705
Mischief,78964
Break and Enter Residential/Other,64425
Other Theft,59749
Offence Against a Person,59015
Theft of Vehicle,40320
Break and Enter Commercial,36952
Theft of Bicycle,29092
Vehicle Collision or Pedestrian Struck (with Injury),24232
Vehicle Collision or Pedestrian Struck (with Fatality),276


Judging from the counts of each kind of crime, it seems that theft is the most prevalent crime.

####Crime Categories

Now that the crime list includes violent offences as well, we should split it up into 5 new categories:
1. Theft
2. Mischief
3. Vehicle Accidents
4. Violent Offences
5. Break and Enter (BNE)

In [22]:
# cut everything except YEAR TYPE COUNT
crimes = full_crime_df.select("YEAR", "TYPE").sort("YEAR")

# category lists
theft = ["Theft from Vehicle", "Other Theft", "Theft of Vehicle", "Theft of Bicycle"]
vehicle = ["Vehicle Collision or Pedestrian Struck (with Injury)", "Vehicle Collision or Pedestrian Struck (with Fatality)"]
violence = ["Offence Against a Person", "Homicide"]
bne = ["Break and Enter Residential/Other", "Break and Enter Commercial"]

# put all crimes into categories (mischief is just by itself)
crimes = crimes.na.replace(theft, "Theft")
crimes = crimes.na.replace(vehicle, "Vehicle Accidents")
crimes = crimes.na.replace(violence, "Violent Offences")
crimes = crimes.na.replace(bne, "BNE")

# reshape the dataframe so that it has the counts of each category split into each year
data_counts = crimes.groupBy(["YEAR", "TYPE"]).count().alias("COUNT")
data_joined = crimes.join(data_counts, ["YEAR", "TYPE"]).dropDuplicates()
reshaped_df = data_joined.groupby('YEAR').pivot('TYPE').max('count')
crimes = reshaped_df.filter(crimes["YEAR"] != "2019").orderBy("YEAR")
display(crimes)

YEAR,BNE,Mischief,Theft,Vehicle Accidents,Violent Offences
2003,10080,6391,28105,1827,3528
2004,9821,5601,28141,1890,3811
2005,8181,5062,25612,2016,3805
2006,8518,5184,22849,1404,4363
2007,7432,4810,19758,1261,4442
2008,6656,5276,18036,1202,4245
2009,5355,4429,17192,1292,3906
2010,4927,4506,15179,1337,3755
2011,4980,4828,13608,1273,3905
2012,4998,4243,14695,1492,3803


#####Judging by the graph above, a few speculations can be made:
- Theft was declining, but since 2011 has been increasing again
- Vehicle Accidents have remained steady since 2003
- BNEs are slowly declining (possibly due to better security in cars and buildings)

####Incorporating Property Values

Now that we have the crimes nice and sorted into categories by year, we can find the average property value price per year and append it to the crime dataframe, setting it up for the linear regressions.

In [25]:
# store the 2006 - 2018 property tax dataframes in a dictionary as {year: df}
property_dict = {}

# read the dataset in from the DBFS, and select only the columns from the older datasets 
# as the newer ones have a few extra columns that we don't need
for year in range(2006, 2018 + 1):
  property_dict[year] = spark.read.csv("/FileStore/tables/property_tax_report{}.csv".format(year), inferSchema=True, header=True).select("TAX_ASSESSMENT_YEAR", "CURRENT_LAND_VALUE", "CURRENT_IMPROVEMENT_VALUE", "PID", "LEGAL_TYPE", "FOLIO", "LAND_COORDINATE", "LOT", "BLOCK", "PLAN", "DISTRICT_LOT", "FROM_CIVIC_NUMBER", "TO_CIVIC_NUMBER", "STREET_NAME", "PROPERTY_POSTAL_CODE", "NARRATIVE_LEGAL_LINE1", "NARRATIVE_LEGAL_LINE2", "NARRATIVE_LEGAL_LINE3", "NARRATIVE_LEGAL_LINE4", "NARRATIVE_LEGAL_LINE5", "YEAR_BUILT", "BIG_IMPROVEMENT_YEAR", "TAX_LEVY", "NEIGHBOURHOOD_CODE")

In [26]:
#combining all of the years
property_df = property_dict[2006].union(property_dict[2007])
for year in range(2006, 2018 + 1):
  if year != 2006:
    property_df = property_df.union(property_dict[year])
    
print(property_dict.keys())

dict_keys([2016, 2017, 2018, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

The current land value and current property value will be added together to get an overall value for each property.

In [28]:
property_df = property_df.withColumn("OVERALL_VALUE", sf.col("CURRENT_LAND_VALUE") + sf.col("CURRENT_IMPROVEMENT_VALUE"))

In [29]:
display(property_df)

TAX_ASSESSMENT_YEAR,CURRENT_LAND_VALUE,CURRENT_IMPROVEMENT_VALUE,PID,LEGAL_TYPE,FOLIO,LAND_COORDINATE,LOT,BLOCK,PLAN,DISTRICT_LOT,FROM_CIVIC_NUMBER,TO_CIVIC_NUMBER,STREET_NAME,PROPERTY_POSTAL_CODE,NARRATIVE_LEGAL_LINE1,NARRATIVE_LEGAL_LINE2,NARRATIVE_LEGAL_LINE3,NARRATIVE_LEGAL_LINE4,NARRATIVE_LEGAL_LINE5,YEAR_BUILT,BIG_IMPROVEMENT_YEAR,TAX_LEVY,NEIGHBOURHOOD_CODE,OVERALL_VALUE
2006,84600,481000,018-392-164,LAND,120590054001,12059005,1,1,LMP11726,185,null,401,BURRARD ST,V6C 3R2,LOT 1 BLOCK 1 PLAN LMP11726 DIST,RICT LOT 185 NEW WESTMINSTER,null,null,null,2002,2002,null,26,565600.0
2006,73353,135830,026-600-439,STRATA,646122720002,64612272,2,null,BCS1732,526,null,1654,7TH AVE W,V6J 1S5,LOT 2 PLAN BCS1732 DISTRICT LOT 5,26 NEW WESTMINSTER GROUP 1.,null,null,null,2006,2006,1179.49,7,209183.0
2006,67600,125177,026-600-447,STRATA,646122720003,64612272,3,null,BCS1732,526,null,1648,7TH AVE W,V6J 1S5,LOT 3 PLAN BCS1732 DISTRICT LOT 5,26 NEW WESTMINSTER GROUP 1.,null,null,null,2006,2006,1086.97,7,192777.0
2006,74072,137162,026-600-455,STRATA,646122720004,64612272,4,null,BCS1732,526,null,1644,7TH AVE W,V6J 1S5,LOT 4 PLAN BCS1732 DISTRICT LOT 5,26 NEW WESTMINSTER GROUP 1.,null,null,null,2006,2006,1191.04,7,211234.0
2006,84140,155805,026-600-463,STRATA,646122720005,64612272,5,null,BCS1732,526,null,1640,7TH AVE W,V6J 1S5,LOT 5 PLAN BCS1732 DISTRICT LOT 5,26 NEW WESTMINSTER GROUP 1.,null,null,null,2006,2006,1352.93,7,239945.0
2006,84140,155805,026-600-471,STRATA,646122720006,64612272,6,null,BCS1732,526,null,1636,7TH AVE W,V6J 1S5,LOT 6 PLAN BCS1732 DISTRICT LOT 5,26 NEW WESTMINSTER GROUP 1.,null,null,null,2006,2006,1352.93,7,239945.0
2006,84140,155805,026-600-480,STRATA,646122720007,64612272,7,null,BCS1732,526,null,1632,7TH AVE W,V6J 1S5,LOT 7 PLAN BCS1732 DISTRICT LOT 5,26 NEW WESTMINSTER GROUP 1.,null,null,null,2006,2006,1352.93,7,239945.0
2006,84140,155805,026-600-498,STRATA,646122720008,64612272,8,null,BCS1732,526,null,1628,7TH AVE W,V6J 1S5,LOT 8 PLAN BCS1732 DISTRICT LOT 5,26 NEW WESTMINSTER GROUP 1.,null,null,null,2006,2006,1352.93,7,239945.0
2006,46025,85227,026-600-501,STRATA,646122720009,64612272,9,null,BCS1732,526,201,1650,7TH AVE W,V6J 5N1,LOT 9 PLAN BCS1732 DISTRICT LOT 5,26 NEW WESTMINSTER GROUP 1.,null,null,null,2006,2006,740.07,7,131252.0
2006,34519,63920,026-600-510,STRATA,646122720010,64612272,10,null,BCS1732,526,202,1650,7TH AVE W,V6J 5N1,LOT 10 PLAN BCS1732 DISTRICT LOT,526 NEW WESTMINSTER GROUP 1.,null,null,null,2006,2006,555.05,7,98439.0


Now all of the other columns besides year and overall value can be removed, and the average for each year can be found

In [31]:
# trim all columns except year and overall
prop_df = property_df.select(["TAX_ASSESSMENT_YEAR", "OVERALL_VALUE"])

In [32]:
from pyspark.sql.functions import mean

prop_df = (prop_df
    .groupBy("TAX_ASSESSMENT_YEAR")
    .agg(mean("OVERALL_VALUE").alias("MEAN"))
    .filter(prop_df["TAX_ASSESSMENT_YEAR"] >= 2006)
    .filter(prop_df["TAX_ASSESSMENT_YEAR"] <= 2019)
    .orderBy("TAX_ASSESSMENT_YEAR", ascending=True))
display(prop_df)

TAX_ASSESSMENT_YEAR,MEAN
2006,723236.621305649
2007,883140.8572400311
2008,1016126.7539754785
2009,1007951.8997503361
2010,1009410.0458366973
2011,1126718.7324790321
2012,1266256.9328822477
2013,1283530.1136879553
2014,1300409.1826707004
2015,1408748.9028200025


Finally, the two dataframes can be combined.

In [34]:
crimes = crimes.filter(crimes["YEAR"] >= 2006).filter(crimes["YEAR"] <= 2018)
prop_df = prop_df.selectExpr("TAX_ASSESSMENT_YEAR as YEAR", "MEAN as MEAN_VAL")
data = crimes.join(prop_df, on=["YEAR"]) #.sort("row_index").drop("row_index")

display(data.orderBy("YEAR", ascending=True))

YEAR,BNE,Mischief,Theft,Vehicle Accidents,Violent Offences,MEAN_VAL
2006,8518,5184,22849,1404,4363,723236.621305649
2007,7432,4810,19758,1261,4442,883140.8572400311
2008,6656,5276,18036,1202,4245,1016126.7539754785
2009,5355,4429,17192,1292,3906,1007951.8997503361
2010,4927,4506,15179,1337,3755,1009410.0458366973
2011,4980,4828,13608,1273,3905,1126718.7324790321
2012,4998,4243,14695,1492,3803,1266256.9328822477
2013,4799,4191,14897,1501,3683,1283530.1136879553
2014,5288,4518,18102,1589,3166,1300409.1826707004
2015,5577,4191,19660,1684,3238,1408748.9028200025


###Linear Regressions

Now we can use PySparks MLlib to perform some linear regressions, trying to see which variables affect what.

In [36]:
# predict the label from the features
feature_columns = data.columns[:-1] # omitting the final column

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")

data_2 = assembler.transform(data)

data_2.show()

+----+----+--------+-----+-----------------+----------------+------------------+--------------------+
YEAR| BNE|Mischief|Theft|Vehicle Accidents|Violent Offences| MEAN_VAL| features|
+----+----+--------+-----+-----------------+----------------+------------------+--------------------+
2007|7432| 4810|19758| 1261| 4442| 883140.8572400311|[2007.0,7432.0,48...|
2018|4422| 5742|23234| 1486| 3156|2228273.8326920653|[2018.0,4422.0,57...|
2015|5577| 4191|19660| 1684| 3238|1408748.9028200025|[2015.0,5577.0,41...|
2006|8518| 5184|22849| 1404| 4363| 723236.621305649|[2006.0,8518.0,51...|
2013|4799| 4191|14897| 1501| 3683|1283530.1136879553|[2013.0,4799.0,41...|
2014|5288| 4518|18102| 1589| 3166|1300409.1826707004|[2014.0,5288.0,45...|
2012|4998| 4243|14695| 1492| 3803|1266256.9328822477|[2012.0,4998.0,42...|
2009|5355| 4429|17192| 1292| 3906|1007951.8997503361|[2009.0,5355.0,44...|
2016|5681| 4599|22633| 1721| 3206|1622112.4676377024|[2016.0,5681.0,45...|
2010|4927| 4506|15179| 1337| 3755|1009410.0458366973|[2010.0,4927.0,45...|
2011|4980| 4828|13608| 1273| 3905|1126718.7324790321|[2011.0,4980.0,48...|
2008|6656| 5276|18036| 1202| 4245|1016126.7539754785|[2008.0,6656.0,52...|
2017|4662| 5393|22165| 1533| 3244|2086499.3656747644|[2017.0,4662.0,53...|
+----+----+--------+-----+-----------------+----------------+------------------+--------------------+

In [37]:
#split the data into a train set and a test set
train, test = data_2.randomSplit([0.7, 0.3])

In [38]:
# training the learning algorithm
from pyspark.ml.regression import LinearRegression
algo = LinearRegression(featuresCol="features", labelCol="MEAN_VAL")
model = algo.fit(train)

In [39]:
# evaluation of the model's performance
evaluation_summary = model.evaluate(test)
print("Mean absolute error: ", evaluation_summary.meanAbsoluteError)
print("\nRoot mean squared error: ", evaluation_summary.rootMeanSquaredError)
print("\nR2: ", evaluation_summary.r2)

Mean absolute error: 79684.60113461953

Root mean squared error: 83545.6529129493

R2: 0.6385679950311804

In [40]:
# predicting the test values
predictions = model.transform(test)
predictions.show()

+----+----+--------+-----+-----------------+----------------+------------------+--------------------+------------------+
YEAR| BNE|Mischief|Theft|Vehicle Accidents|Violent Offences| MEAN_VAL| features| prediction|
+----+----+--------+-----+-----------------+----------------+------------------+--------------------+------------------+
2007|7432| 4810|19758| 1261| 4442| 883140.8572400311|[2007.0,7432.0,48...| 763385.1378733814|
2012|4998| 4243|14695| 1492| 3803|1266256.9328822477|[2012.0,4998.0,42...| 1210195.096312642|
2010|4927| 4506|15179| 1337| 3755|1009410.0458366973|[2010.0,4927.0,45...|1070335.8092644513|
2008|6656| 5276|18036| 1202| 4245|1016126.7539754785|[2008.0,6656.0,52...| 934131.6688010097|
+----+----+--------+-----+-----------------+----------------+------------------+--------------------+------------------+

###Future Trends

Now that the model is trained according to the crime rates, lets see how manipulating these variables will affect the future property values.  First, let's see what happens when each crime category drops by 70 each year (+350 total over 5 years)

In [42]:
# values for 2018
year = 2018
bne = 4422
mischief = 5742
theft = 23234
vehicle = 1486
violence = 3156

# multiplier
m = -70

# assume crime rates will drop down by 50 per category per year
rdd = sc.parallelize([(year+1, bne-m, mischief-m, theft-m, vehicle-m, violence-m),
                      (year+2, bne-m*2, mischief-m*2, theft-m*2, vehicle-m*2, violence-m*2),
                      (year+3, bne-m*3, mischief-m*3, theft-m*3, vehicle-m*3, violence-m*3),
                      (year+4, bne-m*4, mischief-m*4, theft-m*4, vehicle-m*4, violence-m*4),
                      (year+5, bne-m*5, mischief-m*5, theft-m*5, vehicle-m*5, violence-m*5)])
future_df = rdd.toDF(["YEAR", "BNE", "Mischief", "Theft", "Vehicle Accidents", "Violent Offences"])

# apply the model to the new df
data_3 = assembler.transform(future_df)
five_years = model.transform(data_3)
display(five_years)

YEAR,BNE,Mischief,Theft,Vehicle Accidents,Violent Offences,features,prediction
2019,4492,5812,23304,1556,3226,"List(1, 6, List(), List(2019.0, 4492.0, 5812.0, 23304.0, 1556.0, 3226.0))",2379805.7814376354
2020,4562,5882,23374,1626,3296,"List(1, 6, List(), List(2020.0, 4562.0, 5882.0, 23374.0, 1626.0, 3296.0))",2516544.690096006
2021,4632,5952,23444,1696,3366,"List(1, 6, List(), List(2021.0, 4632.0, 5952.0, 23444.0, 1696.0, 3366.0))",2653283.598754376
2022,4702,6022,23514,1766,3436,"List(1, 6, List(), List(2022.0, 4702.0, 6022.0, 23514.0, 1766.0, 3436.0))",2790022.5074127316
2023,4772,6092,23584,1836,3506,"List(1, 6, List(), List(2023.0, 4772.0, 6092.0, 23584.0, 1836.0, 3506.0))",2926761.416071117


####Housing Market Increases

When each category is increased by a margin of 70 each year, the housing market rises.

Let's see what happens when the OPPOSITE is applied, when each year's categories go down by 70 each year.

In [44]:
# multiplier
m = 70

# assume crime rates will drop down by 50 per category per year
rdd2 = sc.parallelize([(year+1, bne-m, mischief-m, theft-m, vehicle-m, violence-m),
                      (year+2, bne-m*2, mischief-m*2, theft-m*2, vehicle-m*2, violence-m*2),
                      (year+3, bne-m*3, mischief-m*3, theft-m*3, vehicle-m*3, violence-m*3),
                      (year+4, bne-m*4, mischief-m*4, theft-m*4, vehicle-m*4, violence-m*4),
                      (year+5, bne-m*5, mischief-m*5, theft-m*5, vehicle-m*5, violence-m*5)])
future_df2 = rdd2.toDF(["YEAR", "BNE", "Mischief", "Theft", "Vehicle Accidents", "Violent Offences"])

# apply the model to the new df
data_4 = assembler.transform(future_df2)
five_years2 = model.transform(data_4)
display(five_years2)

YEAR,BNE,Mischief,Theft,Vehicle Accidents,Violent Offences,features,prediction
2019,4352,5672,23164,1416,3086,"List(1, 6, List(), List(2019.0, 4352.0, 5672.0, 23164.0, 1416.0, 3086.0))",2234801.08665812
2020,4282,5602,23094,1346,3016,"List(1, 6, List(), List(2020.0, 4282.0, 5602.0, 23094.0, 1346.0, 3016.0))",2226535.30053699
2021,4212,5532,23024,1276,2946,"List(1, 6, List(), List(2021.0, 4212.0, 5532.0, 23024.0, 1276.0, 2946.0))",2218269.514415875
2022,4142,5462,22954,1206,2876,"List(1, 6, List(), List(2022.0, 4142.0, 5462.0, 22954.0, 1206.0, 2876.0))",2210003.728294745
2023,4072,5392,22884,1136,2806,"List(1, 6, List(), List(2023.0, 4072.0, 5392.0, 22884.0, 1136.0, 2806.0))",2201737.942173615


####Housing market decreases

Now, when the crime rates DECREASE, the property market also goes down with it.

###Speculations and Conclusions

It seems to be that the property market and the crime rates seems to be somewhat related.  As the crime rates rise, so does the market, and when they fall, the market falls as well.  This seems counter-intuitive, but actually makes sense if thought about.

When the housing market is rising, more people can't afford housing, and in turn begin commiting more crimes.
The opposite can also be said, as when the housing market is lowering and more affordable, less people have to turn to crime.

In [47]:
#todo add more test cases to the model to see how multiple situations would work (i.e. higher thefts and lower BNE)